In [1]:
!pip install torchvision

In [2]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=4c9eb3fa1c697641b2c103ea41b80cb663061fdd94e7b2d371389ad15e667fbe
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [3]:
import time
import datetime
import os
import copy
import cv2
import random
import numpy as np
import json
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import transforms, datasets
from torch.utils.data import Dataset,DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
import torch.nn.functional as F

In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install wandb
import wandb

In [ ]:
wandb.login()

In [ ]:
# GPU 설정, 배치 사이즈 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
hyper_param_batch = 10

In [ ]:
!wandb sweep '/content/drive/My Drive/ai_test/sweep_config.yaml'

In [ ]:
sweep_id = "lb9cs1tz"

In [ ]:
'''wandb.init(project='fine_crust', entity='circle00', group='circle_team')

wandb.config = {
  "learning_rate": 1e-4,
  "epochs": 10,
  "batch_size": hyper_param_batch
}'''

'wandb.init(project=\'fine_crust\', entity=\'circle00\', group=\'circle_team\')\n\nwandb.config = {\n  "learning_rate": 1e-4,\n  "epochs": 10,\n  "batch_size": hyper_param_batch\n}'

In [ ]:
 #PyTorch의 난수 생성기에 시드 값을 설정
random_seed = 100
random.seed(random_seed)
torch.manual_seed(random_seed)

In [ ]:
#예측할 클래스의 총 수
num_classes = 4
model_name = 'efficientnet-b0'
train_name = 'fine_crust' # 폴더명 바꿔!!!

In [ ]:
PATH = '/content/drive/My Drive/ai_test/scalp_weights_fine_crust/'

In [ ]:
data_train_path = '/content/drive/My Drive/ai_test/train_data/'+train_name+'/train_light/train_light'
data_validation_path = '/content/drive/My Drive/ai_test/train_data/'+train_name+'/validation_light/validation_light'

In [ ]:
# 데이터셋 경로 설정 , 테스트 데이터 경로 설정
data_train_path = '/content/drive/My Drive/ai_test/train_data/'+train_name+'/train'
data_validation_path = '/content/drive/My Drive/ai_test/train_data/'+train_name+'/validation'

In [ ]:
#image_size = EfficientNet.get_image_size(model_name)
#print(image_size)

In [ ]:
#model = EfficientNet.from_pretrained(model_name, num_classes=num_classes)
#model.to('cpu')
#del model
#torch.cuda.empty_cache()

In [ ]:
#pretraines 모델을 device로 이동
#model = EfficientNet.from_pretrained(model_name, num_classes=num_classes)
#model = model.to(device)

In [ ]:
import torchvision.models as models
from torchvision.models import ResNet50_Weights, MobileNet_V2_Weights

# ResNet50 가중치 미리 다운로드 및 분류층 조정
print("Downloading and setting up ResNet50...")
resnet50_weights = ResNet50_Weights.DEFAULT
resnet50 = models.resnet50(weights=resnet50_weights)
resnet50.fc = nn.Linear(resnet50.fc.in_features, 4)  # 4개 클래스로 설정

# EfficientNet-B0 및 B4 (EfficientNet-PyTorch 라이브러리를 사용하므로 별도 변경 불필요)
print("Downloading and setting up EfficientNet-B0...")
efficientnet_b0 = EfficientNet.from_pretrained('efficientnet-b0', num_classes=4)
print("Downloading and setting up EfficientNet-B4...")
efficientnet_b4 = EfficientNet.from_pretrained('efficientnet-b4', num_classes=4)

print("All model weights downloaded and models are set up for 4-class classification.")


Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b4
All model weights downloaded and models are set up for 4-class classification.


In [ ]:
image_size = 224

In [ ]:
from torchvision import transforms
import torchvision.transforms.functional as TF

global image_size

def random_transforms(image):
    # 사용자 정의 변환 함수
    if torch.rand(1) < 0.5:
        image = TF.adjust_gamma(image, gamma=0.9, gain=1)
    if torch.rand(1) < 0.5:
        image = TF.adjust_hue(image, hue_factor=-0.05)
    return image

transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(image_size, scale=(0.5, 1.0)),  # 무작위 크기와 비율로 자르기 범위 확대
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),  # ColorJitter 강도 증가
    transforms.RandomGrayscale(p=0.25),  # 흑백 변환 확률 증가
    transforms.RandomRotation(20),  # 회전 각도 증가
    transforms.RandomAffine(0, shear=15, scale=(0.8, 1.2)),  # Shear 강도 증가
    transforms.RandomPerspective(distortion_scale=0.25, p=0.5),  # 임의의 관점 변형 파라미터 조정
    transforms.Lambda(random_transforms),  # 사용자 정의 람다 변환 적용
    transforms.Resize([int(image_size), int(image_size)], interpolation=4),  # 크기 조정 위치 유지
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
#vaild_data Compose
transforms_val = transforms.Compose([
                                        transforms.Resize([int(224), int(224)], interpolation=4),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                      ])

In [ ]:
#폴더 구조를 기반으로 데이터셋을 자동으로 정렬
train_data_set = datasets.ImageFolder(data_train_path, transform=transforms_train)
val_data_set = datasets.ImageFolder(data_validation_path, transform=transforms_val)

In [ ]:
dataloaders = {}
batch_num = {}

In [ ]:
from torch.utils.data import SubsetRandomSampler

In [ ]:
def get_random_sampler(dataset, percentage):
    num_samples = int(len(dataset) * percentage)
    indices = torch.randperm(len(dataset)).tolist()
    return SubsetRandomSampler(indices[:num_samples])

In [ ]:
#train_sampler = get_random_sampler(train_data_set, 0.9)

In [ ]:
valid_sampler = get_random_sampler(val_data_set, 1)

In [ ]:
batch_num['train'], batch_num['val'] = len(train_data_set), len(val_data_set)
#batch_num['test'] = len(test_data_set)

In [ ]:
print('batch_size : %d,  train/val : %d / %d' % (hyper_param_batch, batch_num['train'], batch_num['val']))

batch_size : 10,  train/val : 12653 / 3639


In [ ]:
#속성들 출력
class_names = train_data_set.classes
print(class_names)

['0', '1', '2', '3']


In [ ]:
!pip install tqdm
from tqdm.auto import tqdm

In [ ]:
global_best_val_acc = 0.0

In [ ]:
def save_model(model, PATH, train_name):
    # .pt 형식으로 모델 및 상태 저장
    model_pt_path = PATH + 'aram_' + train_name + '231121_TEST_1' + '.pt'
    torch.save(model, model_pt_path)
    torch.save(model.state_dict(), PATH + 'president_aram_' + train_name + '231121_TEST_1' + '.pt')
    print('model saved')

    # Wandb에 .pkl 및 .pt 파일 저장
    wandb.save(model_pt_path)

In [ ]:
val_probs = []  # 검증 단계에서 각 배치의 예측 확률을 저장하는 리스트

In [ ]:
# 전역 변수로 최고 검증 정확도 초기화
global_best_val_acc = 0.0
global_sweep_cnt = 1

In [ ]:
import time
import copy
import torch.nn.functional as F
from tqdm import tqdm
from wandb import AlertLevel, alert

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    global global_best_val_acc  # 전역 변수 사용 선언
    global global_sweep_cnt

    start_time = time.time()
    current_best_val_acr = 0.0
    patience = 5  # 조기 중단을 위한 Epoch 허용치
    best_val_loss = float('inf')
    epochs_no_improve = 0

    best_acc = 0.0  # 현재 스윕의 최고 검증 정확도
    best_model_wts = copy.deepcopy(model.state_dict())
    train_loss, train_acc, val_loss, val_acc = [], [], [], []

    val_probs = []  # 검증 단계에서 각 배치의 예측 확률을 저장하는 리스트

    for epoch in range(num_epochs):
        # 에폭 시작 알림
        '''wandb.alert(
            title=f'☆☆☆Epoch {epoch+1}/{num_epochs} Starting☆☆☆',
            text=f'☆☆☆Epoch {epoch+1} of {num_epochs} is starting☆☆☆',
            level=AlertLevel.INFO
        )'''

        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            num_cnt = 0

            progress_bar = tqdm(dataloaders[phase], desc=f'Epoch {epoch}/{num_epochs - 1} - {phase}', leave=False)
            for inputs, labels in progress_bar:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += inputs.size(0)

                progress_bar.set_postfix(loss=running_loss/num_cnt, acc=100. * running_corrects.double() / num_cnt)

                if phase == 'val':
                    probs = F.softmax(outputs, dim=1)
                    val_probs.extend(probs.detach().cpu().numpy())

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / num_cnt
            epoch_acc = running_corrects.double() / num_cnt

            wandb.log({"epoch": epoch, f"{phase}_loss": epoch_loss, f"{phase}_acc": epoch_acc})

            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                val_loss.append(epoch_loss)
                val_acc.append(epoch_acc)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 현재 스윕의 최고 검증 정확도 업데이트
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    print("best model saved")
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                    print("Non-update count: " + str(epochs_no_improve))

                if epochs_no_improve == patience:
                    print("Early stopping")
                    time_elapsed = time.time() - start_time  # 조기 종료 시점의 학습 시간 계산
                    if best_acc > global_best_val_acc:
                      global_best_val_acc = best_acc
                      model.load_state_dict(best_model_wts)
                      save_model(model, PATH, train_name)
                      wandb.alert(
                        title='★★★ EARLY STOPPING ★★★',
                        text=f'★★★NEW UPDATE SAVED! {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s with Best Val Acc: {global_best_val_acc:.4f}★★★',
                        level=AlertLevel.INFO
                      )
                    else:
                      wandb.alert(
                        title='★★★ EARLY STOPPING ★★★',
                        text=f'★★★NO UPDATE... {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s with Best Val Acc: {global_best_val_acc:.4f}★★★',
                        level=AlertLevel.INFO
                      )
                    return model, train_loss, train_acc, val_loss, val_acc, val_probs


                # 에폭이 끝날 때마다 알람 발송
                wandb.alert(
                title=f'☆☆☆Epoch {epoch+1}/{num_epochs} Complete☆☆☆',
                text=f'☆☆☆Epoch {epoch+1} finished with Val Acc: {epoch_acc:.4f} and Val Loss: {epoch_loss:.4f}☆☆☆',
                level=AlertLevel.INFO
                )
    current_best_val_acr = best_acc
    time_elapsed = time.time() - start_time
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    wandb.log({"sweep_best_val_acr":current_best_val_acr})


    # 전체 스윕에서의 최고 검증 정확도와 비교하여 모델 저장
    if best_acc > global_best_val_acc:
        global_best_val_acc = best_acc
        print(f'New global best model saved - Val Acc: {global_best_val_acc:.4f}')
        model.load_state_dict(best_model_wts)
        save_model(model, PATH, train_name)
        wandb.log({"best_val_acc": global_best_val_acc})
        wandb.alert(
          title=f'★★★Training Complete : {global_sweep_cnt} ★★★',
          text=f'★★★NEW UPDATE SAVED! {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s with Best Val Acc: {global_best_val_acc:.4f}★★★',
          level=AlertLevel.INFO
        )
    else:
        print(f'Not the best model, best val Acc: {global_best_val_acc:4f}')
        wandb.alert(
          title=f'★★★Training Complete : {global_sweep_cnt} ★★★',
          text=f'★★★NO UPDATE... {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s with Best Val Acc: {global_best_val_acc:.4f}★★★',
          level=AlertLevel.INFO
        )

    global_sweep_cnt +=1

    return model, train_loss, train_acc, val_loss, val_acc, val_probs

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# 클래스별 샘플 수 계산
#class_counts = [0] * num_classes
#for _, target in train_data_set:
#    class_counts[target] += 1

#print(class_counts)

In [ ]:
class_sample_counts = [534, 4435, 5400, 2284]
total_samples = sum(class_sample_counts)
class_weights = [total_samples / count for count in class_sample_counts]

# PyTorch 텐서로 변환
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

In [ ]:
#criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
criterion = nn.CrossEntropyLoss()


#optimizer_ft = optim.Adam(model.parameters(),lr = 1e-4)
#exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

num_epochs = 30

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=num_epochs)

In [ ]:
def train():

    global image_size

    with wandb.init(group='circle_team_1121_1') as run:
        config = run.config

        hyper_param_batch = config.batch_size

        #모델 선택
        if config.model_name == 'efficientnet-b4':
            image_size = 380
            model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=num_classes)
        elif config.model_name == 'efficientnet-b0':
            image_size = EfficientNet.get_image_size('efficientnet-b0')
            model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_classes)
        elif config.model_name == 'resnet50':
            image_size = 224  # ResNet50의 기본 이미지 크기
            model = models.resnet50(pretrained=True)
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, num_classes)
        elif config.model_name == 'mobilenet_v2':
            image_size = 224  # MobileNetV2의 기본 이미지 크기
            model = models.mobilenet_v2(pretrained=True)
            model.classifier[1] = nn.Linear(model.last_channel, num_classes)
        else:
            raise ValueError(f"Unsupported model type: {config.model_name}")

        model = model.to(device)
        if config.optimizer == 'adam':
            optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
        elif config.optimizer == 'sgd':
            optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay, momentum=0.9)
        else:
            raise ValueError(f"Unsupported optimizer type: {config.optimizer}")

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

        dataloaders['train'] = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True, persistent_workers=True, num_workers=2)
        dataloaders['val'] = DataLoader(val_data_set, batch_size=hyper_param_batch, shuffle=False, persistent_workers=True, sampler=valid_sampler, num_workers=2)


        # train_model 함수 호출
        train_model(model, criterion, optimizer, scheduler, num_epochs)

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
wandb.finish()